### Index 생성

In [10]:
# import library
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain_upstage import ChatUpstage, UpstageEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_core.documents import Document
from dotenv import load_dotenv
import glob
import os

In [11]:
load_dotenv()

index_name = "dev-02"
pc = Pinecone()
llm_upstage = ChatUpstage(api_key=os.environ.get("UPSTAGE_API_KEY"), temperature=0, model="solar-pro")
embeddings_query = UpstageEmbeddings(model="embedding-query") #4096

In [12]:
if index_name not in [index_info["name"] for index_info in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=4096, 
        metric="dotproduct",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        ) 
    )
    print(f"{index_name} has been successfully created")
else:
    print(f"{index_name} is already exists.")

dev-02 is already exists.


In [13]:
index = pc.Index(index_name)

In [14]:
llm = ChatUpstage(api_key=os.environ.get("UPSTAGE_API_KEY"), temperature=0, model="solar-pro")
embeddings_passage = UpstageEmbeddings(model="embedding-passage") #4096
embeddings_query = UpstageEmbeddings(model="embedding-query") #4096
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=150)

split_docs = []
files = sorted(glob.glob("data/*.pdf"))

for file in files:
    loader = PyMuPDFLoader(file)
    split_docs.extend(loader.load_and_split(text_splitter))

# 문서 개수 확인
len(split_docs)

/opt/homebrew/anaconda3/envs/venv/lib/python3.12/site-packages/langchain_community/document_loaders/parsers/pdf.py:322: UserWarning: Warning: Empty content on page 225 of document data/천재교육_고등교과서_지구과학Ⅱ_오필석(15개정)_교과서 본문.pdf
  warnings.warn(


1579

In [15]:
import re

def remove_question_and_specific_sentences(paragraph):
    # 정규표현식으로 "?" 또는 "보자"로 끝나는 문장 제거
    result = re.sub(r'[^.?!]*[\?]|[^.?!]*보자\.', '', paragraph)
    # 여러 개의 공백을 하나로 정리
    result = re.sub(r'\s+', ' ', result).strip()
    return result

for doc in split_docs:
    doc.page_content = remove_question_and_specific_sentences(doc.page_content)

In [19]:
split_docs[1000].page_content

'편광 현미경을 이용하여 광물 관찰하기 ➊ 재물대 위에 암석 박편을 올려놓고 광원을 켠다. ➋ 광물 A를 상부 편광판을 뺀 상태(개방 니콜)에서 관 찰하고, 광물의 광학적 성질(투명성, 색, 다색성, 입자 크기, 쪼개짐 등)을 기록한다. ➌ 개방 니콜 상태에서 재물대를 회전시킬 경우 어떤 변화가 있는지 관찰한다. ➍ 상부 니콜을 끼운 상태(직교 니콜)에서 재물대를 회 전시킬 경우 일어나는 광물의 광학적 성질(광학적 등 방성과 이방성, 간섭색, 소광 현상 등)을 기록한다. ➎ 광물 B~D에 대해서도 과정 ➋ ~ ➍를 반복한다. ➏ 관찰한 광물 모습을 스케치한다. 문제 해결력'

In [20]:
docsearch = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embeddings_passage)
# docsearch.add_documents(split_docs)

['66f23f00-ceb1-40ae-8edb-827255df2dbe',
 'ca383dd0-d395-4e0e-a90e-e55c404b8129',
 '95e30242-d6cf-4d43-b5b1-b50a1058c5f9',
 '2b70db9d-e25a-44f5-96fe-3ae7f73eb0d4',
 '1182767f-c6bb-40fd-a112-e86fbcd800d1',
 '5074468f-d956-48de-a689-7de7937d5279',
 '7067458b-ee1e-4ef4-8f29-525561f26550',
 '2bc0c9f5-59a1-4b77-a34d-cb26b247f034',
 '2465c144-af71-484e-98be-8c31b1c278a5',
 '3ab1a40c-06ba-4635-bd5b-61aabd97bf34',
 '74e77094-c7bb-486a-98ec-4a7191e5322f',
 'd56d8909-ee61-46ae-87fd-e071ee96a2e0',
 '8f4d6cfc-2778-4468-b7ba-033f9e43e760',
 'be1f7b47-a9f4-4f7c-b7fc-ded44029e6df',
 '5c732d8f-9f4d-4a2a-9e08-e46500ee5450',
 '5e7ab848-ba6e-4cad-bbee-995614221b26',
 '6f3cd196-8ef5-428b-868d-bee506ff3e22',
 'ddfe5d6d-bc5c-45e6-96b4-a742d46281c9',
 '78ba4e0d-7bd0-4366-ad19-8f9acf5e46b5',
 '73e40285-36c1-4c1e-b4c3-c5c65a52653f',
 '47b8eedb-70ee-4877-9e92-da10b4be0f3f',
 '7bc5d99d-0aff-4d55-9f21-4d3719d9bee6',
 'bbd38b84-2a2b-4576-8e63-498f00547e63',
 '200a9891-da39-4e65-9931-504d1a9b008b',
 '3b6be7ce-9fab-

### 테스트 데이터 만들기

In [21]:
split_docs = []
files = sorted(glob.glob("test_data/*.pdf"))

for file in files:
    loader = PyMuPDFLoader(file)
    split_docs.extend(loader.load_and_split(text_splitter))

# 문서 개수 확인
len(split_docs)

832

In [ ]:
### 지구과학 2
from langchain_community.document_loaders import PyMuPDFLoader
import re

def extract_explanations(pdf_path, output_path):
    """
    PDF에서 "해설 | "로 시작하고 마침표로 끝나는 문장을 추출하여 파일로 저장하는 함수
    
    Args:
        pdf_path (str): 입력 PDF 파일 경로
        output_path (str): 출력 텍스트 파일 경로
    """
    # PDF 로드
    loader = PyMuPDFLoader(pdf_path)
    pages = loader.load()
    start_word = "해설"
    end_word = "ㄱ"
    pattern = rf"{start_word}.*?{end_word}\."
    matches = []
    for page in pages:
        matches.extend(re.findall(pattern, page.page_content, re.DOTALL))
        
    matches = [re.sub(rf"\s*{end_word}\.$", "", match) for match in matches]
    matches = [s[s.index(start_word):] for s in matches]
    
    hint_patterns = [r"\d", r"㉠", r"[a-zA-Z]", r"\(가\)", r"\(나\)"]
    explanations = []
    for match in matches:
        if not any(re.search(pattern, match) for pattern in hint_patterns):
            explanations.append(match)

    # 결과 저장
    with open(output_path, 'w', encoding='utf-8') as f:
        for explanation in explanations:
            f.write(explanation + '\n\n')
    
    return len(explanations)

pdf_path = glob.glob("test_data/*.pdf")
output_path = "지구과학.txt"  # 저장할 파일 경로

count = extract_explanations(pdf_path[0], output_path)
print(f"총 {count}개의 해설을 추출하여 {output_path}에 저장했습니다.")

총 171개의 해설을 추출하여 output.txt에 저장했습니다.


In [112]:
### 정치와 법
from langchain_community.document_loaders import PyMuPDFLoader
import re

def extract_explanations(pdf_path, output_path):
    """
    PDF에서 "해설 | "로 시작하고 마침표로 끝나는 문장을 추출하여 파일로 저장하는 함수
    
    Args:
        pdf_path (str): 입력 PDF 파일 경로
        output_path (str): 출력 텍스트 파일 경로
    """
    # PDF 로드
    loader = PyMuPDFLoader(pdf_path)
    pages = loader.load()
    start_word = "정답 찾기"
    end_word = "오답 피하기"
    pattern = rf"{start_word}.*?{end_word}"
    matches = []
    for page in pages:
        matches.extend(re.findall(pattern, page.page_content, re.DOTALL))

    matches = [s[len(start_word):-len(end_word)] for s in matches]
    
    hint_patterns = [r"\d", r"㉠",  r"[a-zA-Z]", r"\(가\)", r"\(나\)",r"ㄱ.",r"ㄴ.",r"ㄷ.",r"ㄹ.",]
    explanations = []
    for match in matches:
        # hint_patterns에 해당하는 패턴이 없는 경우에만 추가
        if not any(re.search(pattern, match) for pattern in hint_patterns):
            explanations.append(match)
    
    # r"①", r"②", r"③", r"④", r"⑤"는 제거
    explanations = [re.sub(r"①|②|③|④|⑤", "", explanation) for explanation in explanations]
    
    # 결과 저장
    with open(output_path, 'w', encoding='utf-8') as f:
        for explanation in explanations:
            f.write(explanation)
    
    return len(explanations)

pdf_path = glob.glob("test_data/*.pdf")
output_path = "정치와법.txt"  # 저장할 파일 경로

count = extract_explanations(pdf_path[0], output_path)
print(f"총 {count}개의 해설을 추출하여 {output_path}에 저장했습니다.")

총 114개의 해설을 추출하여 정치와법.txt에 저장했습니다.


### Ground Truth 생성

In [113]:
data = []
with open(output_path, 'r', encoding='utf-8') as f:
    data = f.read().split("\n\n")

In [114]:
from typing import List
from pydantic import BaseModel, Field
import pandas as pd
from tqdm import tqdm
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

class RAGEvalOutput(BaseModel):
    """틀린 문장과 피드백을 포함하는 출력 형식"""
    modified_statement: str = Field(description="원본 텍스트의 핵심 개념을 포함한 문장 또는 틀린 문장")
    feedback: str = Field(description="문장의 오류가 있는지 여부 (O, X)")

def create_rag_evaluation_dataset(docsearch, num_samples=None):
    """
    Pinecone 인덱스에서 문서를 가져와 RAG 평가용 데이터셋을 생성합니다.
    LangChain을 사용하여 틀린 문장과 피드백을 한 번의 API 호출로 생성합니다.
    
    Args:
        docsearch: Pinecone 벡터 스토어 객체
        num_samples: 생성할 샘플 수 (None일 경우 전체 문서 사용)
    
    Returns:
        생성된 데이터프레임
    """
    # LangChain 컴포넌트 설정
    parser = JsonOutputParser(pydantic_object=RAGEvalOutput)
    
    prompt = ChatPromptTemplate.from_template("""다음 텍스트의 핵심 개념을 파악하고, 
    1) 개념을 활용한 문장을 4개 적어주세요. 이 때, 절반은 맞는 문장, 절반은 틀린 문장으로 작성해주세요.
    2) 개념을 활용한 문장 4개에 대한 정답을 O, X로 표시해주세요
    
    모든 문장은 다음 텍스트의 핵심 개념을 포함해야 합니다.
    모든 문장은 원본 텍스트와 다른 형태여야 합니다.
    
    원본 텍스트:
    {text}
    
    아래 형식으로 응답해주세요:
    {format_instructions}
    """)
    
    # ChatOpenAI 모델 초기화
    model = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
    
    # 체인 구성
    chain = prompt | model | parser
    
    evaluation_data = []
    docsearch = docsearch[:num_samples] if num_samples else docsearch
    for doc in tqdm(docsearch, desc="데이터셋 생성 중"):
        chunk_text = doc
            
        # LangChain을 사용하여 틀린 문장과 피드백 한 번에 생성
        result = chain.invoke({
            "text": chunk_text,
            "format_instructions": parser.get_format_instructions()
        })
        try:
            for res in result:
                evaluation_data.append({
                    'context': chunk_text,
                    'question': res["modified_statement"],
                    'answer': res["feedback"],
                })
        except:
            pass
    # DataFrame 생성
    df = pd.DataFrame(evaluation_data)
    
    # CSV 파일로 저장
    df.to_csv('rag_evaluation_dataset.csv', index=False)
    
    return df


evaluation_df = create_rag_evaluation_dataset(data)
print(f"생성된 데이터셋 크기: {len(evaluation_df)}")
print("\n데이터셋 샘플:")
print(evaluation_df.head())

데이터셋 생성 중:   0%|          | 0/114 [00:00<?, ?it/s]

데이터셋 생성 중: 100%|██████████| 114/114 [06:52<00:00,  3.62s/it]

생성된 데이터셋 크기: 456

데이터셋 샘플:
                                             context  \
0  \n 정치를 넓은 의미로 바라보는 관점은 복잡하고 다\n원화된 현대 사회의 정치 현...   
1  \n 정치를 넓은 의미로 바라보는 관점은 복잡하고 다\n원화된 현대 사회의 정치 현...   
2  \n 정치를 넓은 의미로 바라보는 관점은 복잡하고 다\n원화된 현대 사회의 정치 현...   
3  \n 정치를 넓은 의미로 바라보는 관점은 복잡하고 다\n원화된 현대 사회의 정치 현...   
4   대의 민주제는 주권자인 국민이 선거를 통해 대표\n를 선출하고 선출된 대표가 국민...   

                                            question answer  
0  정치를 좁은 의미로 이해하는 것은 현대 사회의 복잡한 정치 현상을 설명하는 데 적합하다.      X  
1       현대 사회의 정치 현상은 넓은 의미의 정치 관점을 통해 더 잘 이해될 수 있다.      O  
2          정치의 복잡성을 간과하면 현대 사회의 정치 현상을 제대로 설명할 수 없다.      O  
3                         정치 현상은 단순한 관점에서만 설명될 수 있다.      X  
4                      대의 민주제는 국민이 직접 정책을 결정하는 제도이다.      X  


### Solar LLM 테스트 (without RAG)

In [124]:
# 과학 csv 불러오기
science = pd.read_csv("지구과학OX.csv")
# question, answer 컬럼만 추출
science = science[['question', 'answer']]
# 순서 섞기
science = science.sample(frac=1).reset_index(drop=True)
len(science)

688

In [120]:
# science에 대한 OX 문제를 upstage api llm으로 답변 생성
from langchain_upstage import ChatUpstage
prompt = """
    다음 문장이 사실인지 판단해주세요.
    문장: {question}
    답변은 무조건 O 또는 X로만 작성해주세요.
"""
llm_upstage = ChatUpstage(api_key=os.environ.get("UPSTAGE_API_KEY"), temperature=0, model="solar-pro")


def generate_science_answer(question):
    return llm_upstage(prompt.format(question=question)).content

In [ ]:
science['generated_answer'] = science['question'].apply(generate_science_answer)
# 저장
science.to_csv("지구과학OX_upstage.csv", index=False)
# 정답 Percentage
print(f"정답 비율: {science[science['answer'] == science['generated_answer']].shape[0] / science.shape[0] * 100:.5f}%")

,question,answer,generated_answer
0,해풍은 맑은 날에 육지와 바다의 온도 차이로 인해 발생하는 바람이다.,O,O
1,공기가 상승하면서 온도와 이슬점이 같아지면 구름이 생성된다.,O,O
2,수압 경도력은 수압이 높은 곳에서 낮은 곳으로 이동하는 힘이다.,O,O
3,황도에 위치한 별들은 곡선 운동을 한다.,X,O
4,금성은 해가 지기 직후 서쪽 하늘에서 관찰된다.,O,X


In [122]:
science

,question,answer,generated_answer
0,해풍은 맑은 날에 육지와 바다의 온도 차이로 인해 발생하는 바람이다.,O,O
1,공기가 상승하면서 온도와 이슬점이 같아지면 구름이 생성된다.,O,O
2,수압 경도력은 수압이 높은 곳에서 낮은 곳으로 이동하는 힘이다.,O,O
3,황도에 위치한 별들은 곡선 운동을 한다.,X,O
4,금성은 해가 지기 직후 서쪽 하늘에서 관찰된다.,O,X
5,사암은 퇴적암의 일종이다.,O,O
6,우리은하는 중심 부근에서 태양 주위를 회전한다.,X,X
7,구상 성단은 수십 개의 별들이 허술하게 모여 있는 집단이다.,X,X
8,모든 천체는 동일한 방법으로 거리를 측정할 수 있다.,X,X
9,연직 방향의 기압이 중력보다 항상 크면 정역학 평형이 유지된다.,X,X


In [ ]:
from langchain_upstage import ChatUpstage
import pandas as pd
import os
from typing import List
import time

def create_batch_prompt(questions: List[str], batch_size: int = 5) -> str:
    """여러 질문을 하나의 프롬프트로 만듭니다."""
    numbered_questions = [f"{i+1}. {q}" for i, q in enumerate(questions)]
    questions_text = "\n".join(numbered_questions)
    
    return f"""
    다음 문장들이 사실인지 각각 판단해주세요.
    각 문장에 대해 O 또는 X로만 답변해주세요.
    답변 형식: 숫자. O 또는 X
    
    {questions_text}
    """

def parse_batch_response(response: str, batch_size: int) -> List[str]:
    """LLM의 응답을 개별 답변으로 파싱합니다."""
    try:
        # 응답을 줄 단위로 분리하고 불필요한 공백 제거
        lines = [line.strip() for line in response.split('\n') if line.strip()]
        # 'n. O' 또는 'n. X' 형식에서 O/X만 추출
        answers = []
        for line in lines:
            if '.' in line and line.split('.')[-1].strip() in ['O', 'X']:
                answers.append(line.split('.')[-1].strip())
        return answers[:batch_size]  # batch_size만큼만 반환
    except Exception as e:
        print(f"응답 파싱 중 오류 발생: {e}")
        return ['-'] * batch_size  # 오류 시 기본값 반환

def process_questions_in_batches(df: pd.DataFrame, batch_size: int = 5, retry_delay: int = 1) -> pd.DataFrame:
    """데이터프레임의 질문들을 배치로 처리합니다."""
    llm_upstage = ChatUpstage(
        api_key=os.environ.get("UPSTAGE_API_KEY"),
        temperature=0,
        model="solar-pro"
    )
    
    all_answers = []
    questions = df['question'].tolist()
    
    for i in range(0, len(questions), batch_size):
        batch_questions = questions[i:i + batch_size]
        prompt = create_batch_prompt(batch_questions, batch_size)
        
        max_retries = 3
        for attempt in range(max_retries):
            try:
                response = llm_upstage(prompt).content
                batch_answers = parse_batch_response(response, len(batch_questions))
                all_answers.extend(batch_answers)
                print(f"배치 {i//batch_size + 1} 처리 완료: {len(batch_questions)}개 질문")
                break
            except Exception as e:
                print(f"배치 처리 중 오류 발생 (시도 {attempt + 1}/{max_retries}): {e}")
                if attempt < max_retries - 1:
                    time.sleep(retry_delay)
                else:
                    all_answers.extend(['-'] * len(batch_questions))  # 오류 시 기본값
        
        time.sleep(retry_delay)  # API 요청 간 딜레이
    
    df_result = df.copy()
    df_result['generated_answer'] = all_answers
    return df_result

배치 1 처리 완료: 10개 질문
배치 2 처리 완료: 10개 질문
배치 3 처리 완료: 10개 질문
배치 4 처리 완료: 10개 질문
배치 5 처리 완료: 10개 질문
배치 6 처리 완료: 10개 질문
배치 7 처리 완료: 10개 질문
배치 8 처리 완료: 10개 질문
배치 9 처리 완료: 10개 질문
배치 10 처리 완료: 10개 질문
배치 11 처리 완료: 10개 질문
배치 12 처리 완료: 10개 질문
배치 13 처리 완료: 10개 질문
배치 14 처리 완료: 10개 질문
배치 15 처리 완료: 10개 질문
배치 16 처리 완료: 10개 질문
배치 17 처리 완료: 10개 질문
배치 18 처리 완료: 10개 질문
배치 19 처리 완료: 10개 질문
배치 20 처리 완료: 10개 질문
배치 21 처리 완료: 10개 질문
배치 22 처리 완료: 10개 질문
배치 23 처리 완료: 10개 질문
배치 24 처리 완료: 10개 질문
배치 25 처리 완료: 10개 질문
배치 26 처리 완료: 10개 질문
배치 27 처리 완료: 10개 질문
배치 28 처리 완료: 10개 질문
배치 29 처리 완료: 10개 질문
배치 30 처리 완료: 10개 질문
배치 31 처리 완료: 10개 질문
배치 32 처리 완료: 10개 질문
배치 33 처리 완료: 10개 질문
배치 34 처리 완료: 10개 질문
배치 35 처리 완료: 10개 질문
배치 36 처리 완료: 10개 질문
배치 37 처리 완료: 10개 질문
배치 38 처리 완료: 10개 질문
배치 39 처리 완료: 10개 질문
배치 40 처리 완료: 10개 질문
배치 41 처리 완료: 10개 질문
배치 42 처리 완료: 10개 질문
배치 43 처리 완료: 10개 질문
배치 44 처리 완료: 10개 질문
배치 45 처리 완료: 10개 질문
배치 46 처리 완료: 10개 질문
배치 47 처리 완료: 10개 질문
배치 48 처리 완료: 10개 질문
배치 49 처리 완료: 10개 질문
배치 50 처리 완료: 10개 질문
배치 51 처리 

In [ ]:
# CSV 파일 읽기
science = pd.read_csv("지구과학OX.csv")  # 파일명은 실제 파일명으로 수정하세요
# science = science.head(10)  # 일부 데이터만 사용
# 배치 처리 실행
batch_size = 10  # 한 번에 처리할 질문 수
processed_df = process_questions_in_batches(science, batch_size)

# 결과 저장
processed_df.to_csv("지구과학OX_upstage_batch.csv", index=False)

# Error 비율
error_rate = (processed_df['generated_answer'] == '-').mean() * 100
print(f"Error 비율: {error_rate:.5f}%")
# Error row 제거
processed_df = processed_df[processed_df['generated_answer'] != '-']

# 정확도 계산
accuracy = (processed_df['answer'] == processed_df['generated_answer']).mean() * 100
print(f"정답 비율: {accuracy:.5f}%")

Solar-pro: 73.98256%
2분 47초

### Solar LLM with RAG

In [134]:
from langchain_upstage import ChatUpstage, UpstageEmbeddings
from langchain_pinecone import PineconeVectorStore
import pandas as pd
import os
from typing import List
import time

class RAGScienceQA:
    def __init__(
        self,
        index_name: str,
        embedding_model,
        batch_size: int = 5,
        k_similar: int = 3
    ):
        self.batch_size = batch_size
        self.k_similar = k_similar
        
        # Initialize LLM
        self.llm = ChatUpstage(
            api_key=os.environ.get("UPSTAGE_API_KEY"),
            temperature=0,
            model="solar-pro"
        )
        
        # Initialize vector store
        self.vector_store = PineconeVectorStore.from_existing_index(
            index_name=index_name,
            embedding=embedding_model
        )

    def get_relevant_context(self, question: str) -> str:
        """주어진 질문과 관련된 컨텍스트를 검색합니다."""
        similar_docs = self.vector_store.similarity_search(
            question,
            k=self.k_similar
        )
        
        # 검색된 문서들을 하나의 컨텍스트로 결합
        context = "\n".join([doc.page_content for doc in similar_docs])
        return context

    def create_rag_prompt(self, questions: List[str]) -> str:
        """RAG 기반 프롬프트를 생성합니다."""
        # 각 질문에 대한 컨텍스트 검색
        question_contexts = []
        for i, q in enumerate(questions, start=1):
            context = self.get_relevant_context(q)
            question_contexts.append(f"Q{i}: {q}\nC{i}: {context}")
            
        # 번호가 매겨진 질문과 컨텍스트 생성
        qa_text = "\n\n".join(question_contexts)
        
        return f"""
        주어진 각 질문(Q)에 대해 관련 정보(C)를 참고하여 사실 여부를 판단해주세요.
        응답 형식: Q1: O 또는 X, Q2: O 또는 X, ..., Qn: O 또는 X
        
        {qa_text}
        """

    def parse_batch_response(self, response: str, batch_size: int) -> List[str]:
        """LLM의 응답을 개별 답변으로 파싱합니다."""
        try:
            pattern = r"Q(\d+):\s*(O|X)"
            matches = re.findall(pattern, response)
            answer = [ans for _, ans in matches if ans in ['O', 'X']]
            if len(answer) < batch_size:
                print(f"응답 개수({len(answer)})가 배치 크기({batch_size})보다 작습니다.")
                raise ValueError("응답 개수 부족")
            return answer[:batch_size]
        except Exception as e:
            print(f"응답 파싱 중 오류 발생: {e}")
            return ['-'] * batch_size

    def process_questions(self, df: pd.DataFrame) -> pd.DataFrame:
        """데이터프레임의 질문들을 RAG를 사용하여 처리합니다."""
        all_answers = []
        questions = df['question'].tolist()
        
        for i in range(0, len(questions), self.batch_size):
            batch_questions = questions[i:i + self.batch_size]
            prompt = self.create_rag_prompt(batch_questions)
            
            max_retries = 3
            for attempt in range(max_retries):
                try:
                    response = self.llm(prompt).content
                    batch_answers = self.parse_batch_response(
                        response,
                        len(batch_questions)
                    )
                    all_answers.extend(batch_answers)
                    print(f"배치 {i//self.batch_size + 1} 처리 완료: {len(batch_questions)}개 질문")
                    break
                except Exception as e:
                    print(f"배치 처리 중 오류 발생 (시도 {attempt + 1}/{max_retries}): {e}")
                    if attempt < max_retries - 1:
                        time.sleep(1)
                    else:
                        all_answers.extend(['-'] * len(batch_questions)) # Error 시 기본값
            
            time.sleep(1)  # API 요청 간 딜레이
        
        df_result = df.copy()
        df_result['generated_answer'] = all_answers
        return df_result

In [135]:
 # 환경 설정
index_name = "dev-02"

# OpenAI 임베딩 모델 초기화 (또는 다른 임베딩 모델 사용)
embeddings_query = UpstageEmbeddings(model="embedding-query") #4096


# RAG 시스템 초기화
rag_qa = RAGScienceQA(
    index_name=index_name,
    embedding_model=embeddings_query,
    batch_size=5,
    k_similar=2
)

# 데이터 로드
science = pd.read_csv("지구과학OX.csv")

# RAG 기반 처리 실행
processed_df = rag_qa.process_questions(science)

# 결과 저장
processed_df.to_csv("지구과학OX_rag_upstage.csv", index=False)

# Error 비율
error_rate = (processed_df['generated_answer'] == '-').mean() * 100
print(f"Error 비율: {error_rate:.5f}%")
# Error row 제거
processed_df = processed_df[processed_df['generated_answer'] != '-']

# 정확도 계산
accuracy = (processed_df['answer'] == processed_df['generated_answer']).mean() * 100
print(f"정답 비율: {accuracy:.5f}%")

배치 1 처리 완료: 5개 질문
배치 2 처리 완료: 5개 질문
배치 3 처리 완료: 5개 질문
배치 4 처리 완료: 5개 질문
배치 5 처리 완료: 5개 질문
배치 6 처리 완료: 5개 질문
배치 7 처리 완료: 5개 질문
배치 8 처리 완료: 5개 질문
배치 9 처리 완료: 5개 질문
배치 10 처리 완료: 5개 질문
배치 11 처리 완료: 5개 질문
배치 12 처리 완료: 5개 질문
배치 13 처리 완료: 5개 질문
배치 14 처리 완료: 5개 질문
배치 15 처리 완료: 5개 질문
배치 16 처리 완료: 5개 질문
배치 17 처리 완료: 5개 질문
배치 18 처리 완료: 5개 질문
배치 19 처리 완료: 5개 질문
배치 20 처리 완료: 5개 질문
배치 21 처리 완료: 5개 질문
배치 22 처리 완료: 5개 질문
배치 23 처리 완료: 5개 질문
배치 24 처리 완료: 5개 질문
배치 25 처리 완료: 5개 질문
배치 26 처리 완료: 5개 질문
배치 27 처리 완료: 5개 질문
배치 28 처리 완료: 5개 질문
배치 29 처리 완료: 5개 질문
배치 30 처리 완료: 5개 질문
배치 31 처리 완료: 5개 질문
배치 32 처리 완료: 5개 질문
배치 33 처리 완료: 5개 질문
배치 34 처리 완료: 5개 질문
배치 35 처리 완료: 5개 질문
배치 36 처리 완료: 5개 질문
배치 37 처리 완료: 5개 질문
배치 38 처리 완료: 5개 질문
배치 39 처리 완료: 5개 질문
배치 40 처리 완료: 5개 질문
배치 41 처리 완료: 5개 질문
배치 42 처리 완료: 5개 질문
배치 43 처리 완료: 5개 질문
배치 44 처리 완료: 5개 질문
배치 45 처리 완료: 5개 질문
배치 46 처리 완료: 5개 질문
배치 47 처리 완료: 5개 질문
배치 48 처리 완료: 5개 질문
배치 49 처리 완료: 5개 질문
배치 50 처리 완료: 5개 질문
배치 51 처리 완료: 5개 질문
배치 52 처리 완료: 5개 질문
배치 53 처리 완료: 5개 질문
배치

Solar-pro: 73.98256%

Solar-pro(with RAG): 78.63372%